In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [13]:
col_info = pd.read_excel(
    r'https://cdn1.sph.harvard.edu/wp-content/uploads/sites/1268/2012/10/NHEFS_Codebook.xls')
A=['qsmk']
D=['death']
L=['age','sex','race','education','wt71','smokeintensity','smokeyrs','exercise','active']
df = pd.read_csv(
    r'https://cdn1.sph.harvard.edu/wp-content/uploads/sites/1268/1268/20/nhefs.csv',
    usecols=A+D+L)

In [4]:
df.head()

,qsmk,death,sex,age,race,education,wt71,smokeintensity,smokeyrs,active,exercise
0,0,0,0,42,1,1,79.04,30,29,0,2
1,0,0,0,36,0,2,58.63,20,24,0,0
2,0,0,1,56,1,2,56.81,20,26,0,2
3,0,1,0,68,1,1,59.42,3,53,1,2
4,0,0,0,40,0,2,87.09,20,19,1,1


# Bước 1: Tìm $Pr[A|L]$

Bình phương các biến định lượng

In [14]:
continuous = ['age', 'wt71','smokeintensity','smokeyrs']
categorical = ['sex', 'race','education','active','exercise']
quadratic = [x+'_2' for x in continuous]
for q,c in zip(quadratic, continuous):
    df[q] = df[c]**2

Standardize các biến định lượng

In [17]:
scalers = dict()
for col in continuous + quadratic:
    scalers[col] = StandardScaler()
    df[col] = scalers[col].fit_transform(df[[col]])
df.head()

,qsmk,death,sex,age,race,education,wt71,smokeintensity,smokeyrs,active,exercise,age_2,wt71_2,smokeintensity_2,smokeyrs_2
0,0,0,0,-0.157420,1,1,0.507980,0.800732,0.338593,0,2,-0.280586,0.380688,0.542389,0.108418
1,0,0,0,-0.650570,0,2,-0.789973,-0.046716,-0.071434,0,0,-0.700683,-0.743335,-0.259007,-0.281271
2,0,0,1,0.993262,1,2,-0.905714,-0.046716,0.092577,0,2,0.950980,-0.827381,-0.259007,-0.134219
3,0,1,0,1.979561,1,1,-0.739733,-1.487378,2.306721,1,2,2.286672,-0.706028,-0.885698,3.002413
4,0,0,0,-0.321804,0,2,1.019912,-0.046716,-0.481460,1,1,-0.427799,0.915667,-0.259007,-0.597434


In [18]:
m1 = LogisticRegression()
m1.fit(
    X=df[continuous + quadratic + categorical],
    y=df[A].values.ravel())

LogisticRegression()

Calculate IP weight

In [19]:
probs = m1.predict_proba(df[continuous + quadratic + categorical])
df['IP_weight'] = np.reciprocal(np.where(df['qsmk']==0, probs[:,0], probs[:,1]))

In [20]:
df

,qsmk,death,sex,age,race,education,wt71,smokeintensity,smokeyrs,active,exercise,age_2,wt71_2,smokeintensity_2,smokeyrs_2,IP_weight
0,0,0,0,-0.157420,1,1,0.507980,0.800732,0.338593,0,2,-0.280586,0.380688,0.542389,0.108418,1.117128
1,0,0,0,-0.650570,0,2,-0.789973,-0.046716,-0.071434,0,0,-0.700683,-0.743335,-0.259007,-0.281271,1.204552
2,0,0,1,0.993262,1,2,-0.905714,-0.046716,0.092577,0,2,0.950980,-0.827381,-0.259007,-0.134219,1.177777
3,0,1,0,1.979561,1,1,-0.739733,-1.487378,2.306721,1,2,2.286672,-0.706028,-0.885698,3.002413,1.710404
4,0,0,0,-0.321804,0,2,1.019912,-0.046716,-0.481460,1,1,-0.427799,0.915667,-0.259007,-0.597434,1.411743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,0,0,0,0.089154,0,1,-0.508887,1.648180,0.338593,0,0,-0.046301,-0.528188,1.664342,0.108418,1.189021
1625,0,0,1,0.253538,0,1,-0.847843,-0.046716,0.502604,0,0,0.118865,-0.785689,-0.259007,0.284881,1.159054
1626,0,0,1,0.582304,0,3,-0.530509,1.648180,0.420598,0,0,0.470741,-0.545293,1.664342,0.195179,1.172866
1627,0,0,0,1.979561,0,1,-1.186799,-0.470440,1.732684,1,1,2.286672,-1.020462,-0.539495,1.983339,1.750136


In [12]:
df['IP_weight'].describe()

count    1629.000000
mean        1.989756
std         1.372395
min         1.058247
25%         1.244266
50%         1.394110
75%         2.010936
max        14.661135
Name: IP_weight, dtype: float64

In [1]:
_

''